In [1]:
import h5py
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import imageio
from IPython.display import Image
def dice_score_gt(niom,mem,length):
    dice_scores=[]
    size=64
    kernel = np.ones((5,5),np.uint8)
    for i in range(length):
        niom_frame=niom['F4'][i,:,:]
        niom_frame=cv2.resize(niom_frame,(size,size),interpolation=cv2.INTER_CUBIC)
        niom_frame=cv2.GaussianBlur(niom_frame, (7, 7), 0)
        niom_frame = cv2.normalize(niom_frame,  np.zeros((size,size)), 0, 255, cv2.NORM_MINMAX)
        ret, niom_frame = cv2.threshold(niom_frame.astype(np.uint8), 0, 255, cv2.THRESH_OTSU)
        niom_frame=cv2.GaussianBlur(niom_frame, (7, 7), 0)
        niom_frame = cv2.erode(niom_frame, kernel, iterations=1)
        niom_mask=cv2.GaussianBlur(niom_frame, (7, 7), 0)
        ret, niom_mask = cv2.threshold(niom_mask.astype(np.uint8), 0, 255, cv2.THRESH_OTSU)

        mem_frame=mem['F4'][i,:,:]
        mem_frame=cv2.resize(mem_frame,(size,size),interpolation=cv2.INTER_CUBIC)
        mem_frame = cv2.GaussianBlur(mem_frame, (7, 7), 0)
        mem_frame = cv2.normalize(mem_frame, np.zeros((size,size)), 0, 255, cv2.NORM_MINMAX)
        ret, mem_frame = cv2.threshold(mem_frame.astype(np.uint8), 0, 255, cv2.THRESH_OTSU)
        mem_mask = cv2.normalize(mem_frame,  np.zeros((size,size)), 0, 255, cv2.NORM_MINMAX)

        
        intersection = np.sum(mem_mask[niom_mask==255]) * 2.0
        dice = intersection / (np.sum(mem_mask) + np.sum(niom_mask))
        dice_scores.append(dice)
    return np.mean(dice_scores)

In [2]:
def dice_score_pred(niom,predictions,length,channels):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    dice_scores=[]
    size=64
    for i in range(length):
        niom_frame=niom['F4'][i+channels-1,:,:]
        niom_frame=cv2.resize(niom_frame,(size,size),interpolation=cv2.INTER_CUBIC)
        niom_frame=cv2.GaussianBlur(niom_frame, (7, 7), 0)
        niom_frame = cv2.normalize(niom_frame,  np.zeros((size,size)), 0, 255, cv2.NORM_MINMAX)
        ret, niom_frame = cv2.threshold(niom_frame.astype(np.uint8), 0, 255, cv2.THRESH_OTSU)
        niom_frame=cv2.GaussianBlur(niom_frame, (7, 7), 0)
        niom_frame = cv2.erode(niom_frame, kernel, iterations=1)
        niom_mask=cv2.GaussianBlur(niom_frame, (7, 7), 0)
        ret, niom_mask = cv2.threshold(niom_mask.astype(np.uint8), 0, 255, cv2.THRESH_OTSU)
        
        prediction = predictions['predictions'][i,:,:]
        predictions_mask = cv2.normalize(prediction,  np.zeros((size,size)), 0, 255, cv2.NORM_MINMAX)
        
        intersection = np.sum(predictions_mask[niom_mask==255]) * 2.0
        dice = intersection / (np.sum(predictions_mask) + np.sum(niom_mask))
        dice_scores.append(dice)
    return np.mean(dice_scores)

In [ ]:
# from keras import backend as K
def iou_coef(niom, predictions,length, channels,  smooth=1):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    dice_scores=[]
    size=64
    for i in range(length):
        niom_frame=niom['F4'][i+channels-1,:,:]
        niom_frame=cv2.resize(niom_frame,(size,size),interpolation=cv2.INTER_CUBIC)
        niom_frame=cv2.GaussianBlur(niom_frame, (7, 7), 0)
        niom_frame = cv2.normalize(niom_frame,  np.zeros((size,size)), 0, 255, cv2.NORM_MINMAX)
        ret, niom_frame = cv2.threshold(niom_frame.astype(np.uint8), 0, 255, cv2.THRESH_OTSU)
        niom_frame=cv2.GaussianBlur(niom_frame, (7, 7), 0)
        niom_frame = cv2.erode(niom_frame, kernel, iterations=1)
        niom_mask=cv2.GaussianBlur(niom_frame, (7, 7), 0)
        ret, niom_mask = cv2.threshold(niom_mask.astype(np.uint8), 0, 255, cv2.THRESH_OTSU)
        
        prediction = predictions['predictions'][i,:,:]
        predictions_mask = cv2.normalize(prediction,  np.zeros((size,size)), 0, 255, cv2.NORM_MINMAX)

        intersection = np.sum(predictions_mask[niom_mask==255]) * 2.0
        union = np.sum(predictions_mask) + np.sum(niom_mask) - intersection
        iou = np.mean((intersection + smooth) / (union + smooth))
    return iou

In [3]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

from pathlib import Path
if IN_COLAB:
    google.colab.drive.mount("/content/drive")
    
    # Change this if you created the shortcut in a different location
    AUX_DATA_ROOT = Path("/content/drive/MyDrive/BIA2022/")
    
    assert AUX_DATA_ROOT.is_dir(), "Have you forgot to 'Add a shortcut to Drive'?"
    
    import sys
    sys.path.append(str(AUX_DATA_ROOT))
else:
    AUX_DATA_ROOT = Path(".")

Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/BIA2022/data/'
mem_test_sampled = 'mem_sampled_test.h5'
mem_test_full = 'mem_full_test.h5'
mem_full = h5py.File(path+mem_test_full, 'r')
mem_sampled = h5py.File(path+mem_test_sampled, 'r')

niom_test_sampled = 'niom_sampled_test.h5'
niom_test_full = 'niom_full_test.h5'
niom_full = h5py.File(path+niom_test_full, 'r')
niom_sampled = h5py.File(path+niom_test_sampled, 'r')

In [5]:
#GT
print('GT sampled = ',dice_score_gt(niom_full,mem_full,700))

GT sampled =  0.10217779479748584


### UNET

In [ ]:
prediction_sampled = h5py.File('/content/drive/MyDrive/BIA2022/results/predictions__unet_sampled_mask.h5', 'r')
print(dice_score_pred(niom_sampled,prediction_sampled,length = 700,channels = 1))

0.374927448924055


In [ ]:
prediction_sampled = h5py.File('/content/drive/MyDrive/BIA2022/results/predictions__unet_sampled_mask.h5', 'r')
print(iou_coef(niom_sampled,prediction_sampled,length = 700,channels = 1))

0.30921278125080665
